# Using Amazon Redshift Spectrum to Access Data in S3


We can leverage our previously created table in Amazon Athena with its metadata and schema information stored in the AWS Glue Data Catalog to access our data in S3 through Redshift Spectrum. All we need to do is create an external schema in Redshift, point it to our AWS Glue Data Catalog, and point Redshift to the database we’ve created.  


In [1]:
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Connect to Redshift
redshift = boto3.client('redshift')
secretsmanager = boto3.client('secretsmanager')

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

# Set S3 prefixes
tsv_prefix = 'amazon-reviews-pds/tsv'

# Set S3 path to TSV data
s3_path_tsv = 's3://{}/{}'.format(bucket, tsv_prefix)

### Setup Redshift Connection Via SQLAlchemy
The Python SQL Toolkit and Object Relational Mapper (https://pypi.org/project/SQLAlchemy/)

In [2]:
!pip install -q SQLAlchemy==1.3.13

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd

#### Get Redshift credentials

In [4]:
import json

secret = secretsmanager.get_secret_value(SecretId='dsoaws_redshift_login')
cred = json.loads(secret['SecretString'])

master_user_name = cred[0]['username']
master_user_pw = cred[1]['password']

#### Redshift configuration parameters

In [5]:
redshift_cluster_identifier = 'dsoaws'

database_name_redshift = 'dsoaws'
database_name_athena = 'dsoaws'

redshift_port = '5439'

schema_redshift = 'redshift'
schema_athena = 'athena'

table_name_tsv = 'amazon_reviews_tsv'

### Make Sure the Cluster Status is `Available`

In [6]:
response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
print(response['Clusters'][0]['ClusterStatus'])

available


#### Get Redshift endpoint address & IAM Role

In [7]:
redshift_endpoint_address = response['Clusters'][0]['Endpoint']['Address']
iam_role = response['Clusters'][0]['IamRoles'][0]['IamRoleArn']

print(redshift_endpoint_address)
print(iam_role)

dsoaws.cmibhdtvl5bj.us-west-2.redshift.amazonaws.com
arn:aws:iam::903253828154:role/DSOAWS_Redshift


#### Connect to Redshift database engine

In [8]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(master_user_name, master_user_pw, redshift_endpoint_address, redshift_port, database_name_redshift))


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### Configure Session

In [9]:
session = sessionmaker()
session.configure(bind=engine)
s = session()

# Redshift Spectrum
Amazon Redshift Spectrum directly queries data in S3, using the same SQL syntax of Amazon Redshift. You can also run queries that span both the frequently accessed data stored locally in Amazon Redshift and your full datasets stored cost-effectively in S3.

To use Redshift Spectrum, your cluster needs authorization to access data catalog in Amazon Athena and your data files in Amazon S3. You provide that authorization by referencing an AWS Identity and Access Management (IAM) role that is attached to your cluster. 

To use this capability in from your Amazon SageMaker notebook:

* Register your Athena database `dsoaws` with Redshift Spectrum
* Query Your Data in Amazon S3

### Register Athena Database `dsoaws` with Redshift Spectrum to access the data directly in S3 

In [10]:
statement = """
CREATE EXTERNAL SCHEMA IF NOT EXISTS {} FROM DATA CATALOG 
    DATABASE '{}' 
    IAM_ROLE '{}'
    REGION '{}'
    CREATE EXTERNAL DATABASE IF NOT EXISTS
""".format(schema_athena, database_name_athena, iam_role, region_name)

print(statement)


CREATE EXTERNAL SCHEMA IF NOT EXISTS athena FROM DATA CATALOG 
    DATABASE 'dsoaws' 
    IAM_ROLE 'arn:aws:iam::903253828154:role/DSOAWS_Redshift'
    REGION 'us-west-2'
    CREATE EXTERNAL DATABASE IF NOT EXISTS



In [11]:
s.execute(statement)
s.commit()

## Congratulations

So with just one command, we now have access and can query our S3 data lake from Amazon Redshift without moving any data into our data warehouse. This is the power of Redshift Spectrum. 

## Run a sample query

In [12]:
statement = """
SELECT product_category, COUNT(star_rating) AS count_star_rating
    FROM {}.{}
    GROUP BY product_category
    ORDER BY count_star_rating DESC
""".format(schema_athena, table_name_tsv)

print(statement)


SELECT product_category, COUNT(star_rating) AS count_star_rating
    FROM athena.amazon_reviews_tsv
    GROUP BY product_category
    ORDER BY count_star_rating DESC



In [13]:
df = pd.read_sql_query(statement, engine)
df.head(5)

,product_category,count_star_rating
0,Digital_Video_Games,145431
1,Digital_Software,102084


#### But now, let’s actually copy some data from S3 into Amazon Redshift. Let’s pull in customer reviews data from the years 2014 and 2015. 

# Load TSV Data Into Redshift

Create local Redshift tables with Customer Reviews data of each year for the last 2 years

<img src="img/c3-10.png" width="90%" align="left">

### Create `redshift` schema

In [14]:
statement = """CREATE SCHEMA IF NOT EXISTS {}""".format(schema_redshift)

s = session()
s.execute(statement)
s.commit()

## Create Redshift tables for every year

When you create a table, you can specify one or more columns as the **sort key**. Amazon Redshift stores your data on disk in sorted order according to the sort key. This means, you can optimize your table by choosing a sort key that reflects your most frequently used query types. If you query a lot of recent data, you can specify a timestamp column as the sort key. If you frequently query based on range or equality filtering on one column, you should choose that column as the sort key. 

As we are going to run a lot of queries in the next chapter filtering on `product_category`, let’s choose that one as our sort key. 

You can also define a distribution style for every table. When you load data into a table, Redshift distributes the rows of the table among your cluster nodes according to the table’s distribution style. When you run a query, the query optimizer redistributes the rows to the cluster nodes as needed to perform any joins and aggregations. So our goal should be to optimize the rows distribution to minimize needed data movements. There are three distribution styles from which you can choose from: 

KEY distribution - distribute the rows according to the values in one column
ALL distribution - distribute a copy of the entire table to every node
EVEN distribution - the rows are distributed across all nodes in a round-robin-fashion which is the default distribution style

For our table, we’ve chosen **KEY distribution** based on `product_id` as this column has a high cardinality, shows an even distribution and can be used to join with other tables. 

Now we are ready to copy the data from S3 into our new Redshift table. 


In [15]:
# Create table function, pass session, table name prefix and start & end year

def create_redshift_table_tsv(session, table_name_prefix, start_year, end_year):
    for year in range(start_year, end_year + 1, 1):
        current_table_name = table_name_prefix+'_'+str(year)
        statement = """
        CREATE TABLE IF NOT EXISTS redshift.{}( 
             marketplace varchar(2),
             customer_id varchar(8),
             review_id varchar(14),
             product_id varchar(10) DISTKEY,
             product_parent varchar(9),
             product_title varchar(400),
             product_category varchar(24),
             star_rating int,
             helpful_votes int,
             total_votes int,
             vine varchar(1),
             verified_purchase varchar(1),
             review_headline varchar(128),
             review_body varchar(65535),
             review_date varchar(10),
             year int)  SORTKEY (product_category)
        """.format(current_table_name)

        #print(statement)
        session.execute(statement)
    session.commit()
        
    print("Done.")

In [16]:
create_redshift_table_tsv(s, 'amazon_reviews_tsv', 2014, 2015)

Done.


## Insert Data from Athena table into local Redshift table

For such bulk inserts, you can either use a `COPY` command, or an `INSERT INTO` command. In general, the `COPY` command is preferred, as it loads data in parallel and more efficiently from Amazon S3, or other supported data sources. 

If you are loading data or a subset of data from one table into another, you can use the `INSERT INTO` command with a `SELECT` clause for high-performance data insertion. As we’re loading our data from the `athena.amazon_reviews_tsv` table, let’s choose this option. 


In [17]:
# INSERT INTO function, pass session, table name prefix and start & end year

def insert_into_redshift_table_tsv(session, table_name_prefix, start_year, end_year):
    for year in range(start_year, end_year + 1, 1):
        print(year)
        current_table_name = table_name_prefix+'_'+str(year)
        statement = """
            INSERT 
            INTO
                redshift.{}
                SELECT
                    marketplace,
                    customer_id,
                    review_id,
                    product_id,
                    product_parent,
                    product_title,
                    product_category,
                    star_rating,
                    helpful_votes,
                    total_votes,
                    vine,
                    verified_purchase,
                    review_headline,
                    review_body,
                    review_date,
                    CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year
                FROM
                    athena.amazon_reviews_tsv             
                WHERE
                    year = {}
            """.format(current_table_name, year)
        #print(statement)
        session.execute(statement)
        session.commit()        
    print("Done.")

### Note: The following `INSERT INTO` command can take some time to complete. Please be patient. 

In [18]:
insert_into_redshift_table_tsv(s, 'amazon_reviews_tsv', 2014, 2015)

2014
2015
Done.



You might notice that we use a date conversion to parse the year out of our `review_date` column and store it in a separate `year` column which we then use to filter on all records from 2015. This is an example on how you can simplify ETL tasks, as we’re putting our data transformation logic directly in a `SELECT` query and ingest the result into Redshift. 

Another way to optimize our tables would be to create them as a sequence of time-series tables, especially when our data has a fixed retention period. Let’s say we want to store data of the last 2 years (24 months) in our data warehouse, and update with new data once a month. 

If you create one table per month, you can easily remove old data simply by running a `DROP TABLE` command on the corresponding table. This approach is much faster than running a large-scale DELETE process and also saves you from having to run a subsequent VACUUM process to reclaim space and re-sort the rows. 


# Query Redshift

To combine query results across tables, we can use a `UNION ALL` view. Similarly, when we need to delete old data, we remove the dropped table from the UNION ALL view. 

### Use `UNION ALL` across 2 tables (2015, 2014) in our `redshift` schema

In [19]:
statement = """
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
ORDER BY product_category ASC, year DESC
"""

print(statement)


SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
ORDER BY product_category ASC, year DESC



In [20]:
df = pd.read_sql_query(statement, engine)
df.head(20)

,year,product_category,count_star_rating
0,2015,Digital_Software,35585
1,2014,Digital_Software,36745
2,2015,Digital_Video_Games,30026
3,2014,Digital_Video_Games,43754


### Run the same query on our orginal data in S3 / the `athena` schema to verify the results match

In [21]:
statement = """
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year = 2015 OR year = 2014 
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC
"""

print(statement)


SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year = 2015 OR year = 2014 
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC



In [22]:
df = pd.read_sql_query(statement, engine)
df.head(20)

,year,product_category,count_star_rating
0,2015,Digital_Software,35585
1,2014,Digital_Software,36745
2,2015,Digital_Video_Games,30026
3,2014,Digital_Video_Games,43754


## Now Query Across Both Redshift and Athena in a single query

Use `UNION ALL` across 2 Redshift tables (2015, 2014) and the rest from Athena/S3 (2013-1995)

In [23]:
statement = """
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
UNION ALL
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year <= 2013
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC
"""

print(statement)


SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
UNION ALL
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year <= 2013
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC



In [24]:
df = pd.read_sql_query(statement, engine)
df.head(20)

,year,product_category,count_star_rating
0,2015,Digital_Software,35585
1,2014,Digital_Software,36745
2,2013,Digital_Software,20453
3,2012,Digital_Software,5602
4,2011,Digital_Software,2312
5,2010,Digital_Software,1065
6,2009,Digital_Software,292
7,2008,Digital_Software,30
8,2015,Digital_Video_Games,30026
9,2014,Digital_Video_Games,43754


### Let's verify that both Redshift tables and S3 tables got queried by running  `Explain`

In [25]:
statement = """
EXPLAIN SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
UNION ALL
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year <= 2013
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC
"""

print(statement)


EXPLAIN SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
UNION ALL
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year <= 2013
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC



In [26]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

df = pd.read_sql_query(statement, engine)
df.head(100)

,QUERY PLAN
0,XN Merge (cost=1000175871803.46..1000175871873.32 rows=27946 width=1040)
1,"Merge Key: product_category, ""year"""
2,-> XN Network (cost=1000175871803.46..1000175871873.32 rows=27946 width=1040)
3,Send to leader
4,-> XN Sort (cost=1000175871803.46..1000175871873.32 rows=27946 width=1040)
5,"Sort Key: product_category, ""year"""
6,-> XN Append (cost=1148.19..175869739.59 rows=27946 width=1040)
7,"-> XN Subquery Scan ""*SELECT* 1"" (cost=1148.19..1230.22 rows=6562 width=62)"
8,-> XN HashAggregate (cost=1148.19..1164.60 rows=6562 width=62)
9,-> XN Seq Scan on amazon_reviews_tsv_2015 (cost=0.00..656.11 rows=65611 width=62)


Expected Output

```
QUERYPLAN
XN Merge  (cost=1000177373551.14..1000177373584.69 rows=13420 width=1040)
  Merge Key: product_category, year
  ->  XN Network  (cost=1000177373551.14..1000177373584.69 rows=13420 width=1040)
        Send to leader
        ->  XN Sort  (cost=1000177373551.14..1000177373584.69 rows=13420 width=1040)
              Sort Key: product_category, year
              ->  XN Append  (cost=733371.52..177372631.06 rows=13420 width=1040)
                    ->  XN Subquery Scan *SELECT* 1  (cost=733371.52..733372.06 rows=43 width=22)
                          ->  XN HashAggregate  (cost=733371.52..733371.63 rows=43 width=22)
                                ->  XN Seq Scan on amazon_reviews_tsv_2015  (cost=0.00..419069.44 rows=41906944 width=22)
                    ->  XN Subquery Scan *SELECT* 2  (cost=772258.45..772258.98 rows=43 width=23)
                          ->  XN HashAggregate  (cost=772258.45..772258.55 rows=43 width=23)
                                ->  XN Seq Scan on amazon_reviews_tsv_2014  (cost=0.00..441290.54 rows=44129054 width=23)
                    ->  XN Subquery Scan *SELECT* 3  (cost=175866766.67..175867000.02 rows=13334 width=1040)
                          ->  XN HashAggregate  (cost=175866766.67..175866866.68 rows=13334 width=1040)
                                ->  XN S3 Query Scan amazon_reviews_tsv  (cost=175000000.00..175766766.67 rows=13333334 width=1040)
                                      Filter: (date_part_year(to_date((derived_col1)::text, 'YYYY-MM-DD'::text)) <= 2013)
                                      ->  S3 HashAggregate  (cost=175000000.00..175000100.00 rows=40000000 width=1036)
                                            ->  S3 Seq Scan athena.amazon_reviews_tsv location:s3://sagemaker-us-west-2-237178646982/amazon-reviews-pds/tsv format:TEXT  (cost=0.00..100000000.00 rows=10000000000 width=1036)
----- Tables missing statistics: amazon_reviews_tsv_2015, amazon_reviews_tsv_2014 -----
----- Update statistics by running the ANALYZE command on these tables -----
```

# When to use Athena, and when to use Redshift?

### Amazon Athena
Athena should be your preferred choice when running ad-hoc SQL queries on data that is stored in Amazon S3. It doesn’t require you to set up or manage any infrastructure resources, and you don’t need to move any data. It supports structured, unstructured, and semi-structured data. With Athena, you are defining a **“schema on read”** - you basically just log in, create a table and you are good to go. 

### Amazon Redshift
Redshift is targeted for modern data analytics on large sets of structured data. Here, you need to have a predefined **“schema on write”**. Unlike serverless Athena, Redshift requires you to create a cluster (compute and storage resources), ingest the data and build tables before you can start to query, but caters to performance and scale. So for any highly-relational data with a transactional nature (data gets updated), workloads which involve complex joins, and latency requirements to be sub-second, Redshift is the right choice.
